# Lane Detector

In this exercise we will implement the polynomial fitting and then combine all functionality into one `LaneDetector` class


## Setting up Colab

In [ ]:
colab_nb = 'google.colab' in str(get_ipython())

In [ ]:
if colab_nb:
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
if colab_nb:
  %cd /content/drive/My Drive/aad/code/tests/lane_detection

In [ ]:
if colab_nb:
  !pip install segmentation-models-pytorch
  !pip install albumentations --upgrade

## Precompute the grid

In the book you have seen how the tensor `xyp` was computed. Its first two columns have the `x` and `y` values respectively, while the last column has the probability values. The `x` and `y` values will always be the same. Hence they only need to be computed once.
This is what you should implement first. It is marked as "TODO step 3" in `code/exercises/lane_detection/camera_geometry.py`. Note that there is one additional modification to what you have seen in the book: The `cut_v` parameter. In the book the `(x,y,p)` triples were computed from all possible `u, v, p[v,u]`. Here you should restrict yourself to all `v` with `v>cut_v`. The idea is that pixels with low `v` values are too far away or even above the horizon, and hence should not be considered for fitting later. The other modification is of course that you do not need to compute an `xyp` tensor, since you have no probabilities given. You only precompute the first two columns of the `xyp` tensor.

Once you implemented "TODO step 3", check whether your implementation is correct using the unit test:

In [ ]:
# execute this cell to run unit tests on your implementation of step 3
%cd ../../../
!python -m code.tests.lane_detection.camera_geometry_unit_test 3
%cd -

## Implement the LaneDetector class

Your final step is to implement the LaneDetector class. 

1. Read the rest of this notebook. You will find places where it says "TODO" and you are asked to change something. Do not do this yet! For now, you should just see the sample solution at work.
2. Go to `code/exercises/lane_detection/lane_detector.py` and implement the "TODO" items. 
3. Now it is time to test **your** lane detector. Go through all the cells below and execute them. Some cells will have a "TODO". Please resolve them, so that your lane detector is being run.

Does your `LaneDetector` class work to your satisfaction? If not, debug and improve it!

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')
from pathlib import Path
import cv2

In [ ]:
import sys
sys.path.append(str(Path('../../')))
# TODO: In the next two lines, change "solutions" to "exercises". Now your code will be executed here!
from solutions.lane_detection.lane_detector import LaneDetector
from solutions.lane_detection.camera_geometry import CameraGeometry
cg = CameraGeometry()

In [ ]:
image_fn = str(Path("../../../data/Town04_Clear_Noon_09_09_2020_14_57_22_frame_625_validation_set.png").absolute())
image_arr = cv2.imread(image_fn)
image_arr = cv2.cvtColor(image_arr, cv2.COLOR_BGR2RGB)
plt.imshow(image_arr);

### Get lane boundaries from LaneDetector

In [ ]:
# TODO: Change the next line(s), to create an instance of *your* LaneDetector
model_path = Path("../../solutions/lane_detection/fastai_model.pth")
ld = LaneDetector(model_path=model_path)
poly_left, poly_right = ld(image_fn)

In [ ]:
# It should also be possible to pass the image as an array into the lane detector
# The following assertions should not raise an AssertionError
poly_left_2, poly_right_2 = ld(image_arr)
np.testing.assert_allclose(poly_left, poly_left_2, rtol=1e-5)
np.testing.assert_allclose(poly_right, poly_right_2, rtol=1e-5)
# we are using `assert_allclose` to compare floating point numbers here.

In [ ]:
# Let's see how fast the lane detector works:
%timeit poly_left, poly_right = ld(image_arr)

### Get ground truth for lane boundaries

In [ ]:
boundary_fn = image_fn.replace(".png", "_boundary.txt")
boundary_gt = np.loadtxt(boundary_fn)

trafo_fn = image_fn.replace(".png", "_trafo.txt")
trafo_world_to_cam = np.loadtxt(trafo_fn)

In [ ]:
# Map reconstructed left boundary into world reference frame
def map_between_frames(points, trafo_matrix):
    x,y,z = points[:,0], points[:,1], points[:,2]
    homvec = np.stack((x,y,z,np.ones_like(x)))
    return (trafo_matrix @ homvec).T

trafo_world_to_road = cg.trafo_cam_to_road @ trafo_world_to_cam

In [ ]:
left_boundary_3d_gt_world = boundary_gt[:,0:3]

left_boundary_gt_road = map_between_frames(boundary_gt[:,0:3], trafo_world_to_road)
right_boundary_gt_road = map_between_frames(boundary_gt[:,3:], trafo_world_to_road)

### Plot LaneDetector output and ground truth

In [ ]:
# ground truth
plt.plot(left_boundary_gt_road[:,2], -left_boundary_gt_road[:,0], label="ground truth left")
plt.plot(right_boundary_gt_road[:,2], -right_boundary_gt_road[:,0], label="ground truth right")
# LaneDetector
x = np.arange(0,60,1)
yl = poly_left(x)
yr = poly_right(x)
plt.plot(x,yl, ls = "--", label="LaneDector left")
plt.plot(x,yr, ls = "--", label="LaneDector right")
plt.legend()
# TODO: You can also inspect the plot while commenting out the next line
#plt.axis("equal");

In the plot above, the LaneDetector should yield something close to the ground truth (less than 1m error along the y axis). 